In [13]:
# IMPORTS
from scipy import integrate
import pandas as pd
import matplotlib.pyplot as plt
import os, math
from scipy.signal import savgol_filter
from scipy.optimize import curve_fit
import scipy.stats as stats
import numpy as np

# CONSTANTS: THIS CONSTANTS DEFINE WHAT DATA TO BE ANALYZED, THEREFORE WHAT THE PROGRAM WILL DO
interactive = False
if interactive:
    IRRADIATED_DOSIMETERS = True if str(input('As leituras são de dosímetros irradiados? (S/N): '))=='S' else False
    OSL_FILE_NAME = str(input('Nome do arquivo com a resposta osl (com a extensão): '))
    OSL_ROOT_DIR = str(input('Nome relativo da pasta onde estão as resposta osl: '))
else:
    IRRADIATED_DOSIMETERS = True
    OSL_FILE_NAME = 'lado 2.txt'
    OSL_ROOT_DIR = ''

if IRRADIATED_DOSIMETERS:
    IRRADIATED_DOSES = [0.1, 0.3, 0.5, 0.7, 1, 3, 5, 10]
    OSL_RESPONSE_BASE_DIR = ['{} Gy'.format(str(_).replace('.',',')) for _ in IRRADIATED_DOSES]
    LID = False
else:
    IRRADIATED_DOSES = None
    OSL_RESPONSE_BASE_DIR = ['limpos']
    LID = True

In [3]:
class osl_response:
    def __init__(self, file_path=None):
        self.file = file_path
        if file_path != None:
            self.file = os.path.abspath(file_path)   # Verificar se funciona para caminhos com caracters espceiais
            self.dose = os.path.basename(os.path.abspath((os.path.join(os.path.dirname(self.file), '..'))))
            self.pastilha = os.path.basename(os.path.dirname(self.file))
            self.df = pd.read_csv(self.file, header=None, sep=' ', names = ['Tempo (s)', 'Intensidade (u.a.)'])
            self.df = self.df.drop(labels=0, axis=0)
            self.df = self.df.apply(pd.to_numeric)
            self.df['Tempo (s)'] = self.df['Tempo (s)']/1000
            self.Tempo = list(self.df.loc[:,'Tempo (s)'])
            self.Intensidade = list(self.df.loc[:,'Intensidade (u.a.)'])
    
    def savgol_filter(self, a, b):
        if self.file != None:
            _Tempo = self.df.loc[:,'Tempo (s)']
            _Intensidade = savgol_filter(self.df.loc[:,'Intensidade (u.a.)'], a, b)
            return ([_Tempo, _Intensidade])
    
    def total_counts(self):
        if self.file != None:
            _total_counts = integrate.simps(self.df['Intensidade (u.a.)'].values, self.df.loc[:,'Tempo (s)'].values)
            return(_total_counts)
    
    def software_integral(self, integrals_path=None):
        if integrals_path != None:
            try:
                _df = pd.read_csv(integrals_path, header=None, sep=' ', names = ['Dosimeters Names', 'Integrals'], index_col = False)
                return(_df['Integrals'].values.tolist())
            except:
                return('Algo saiu errado. O caminho passado foi:\n\n{}'.format(integrals_path))
        else:
            return('Algo saiu errado. O caminho passado foi:\n\n{}'.format(integrals_path))

class osl_response_group:
    def __init__(self, group_dir, osl_response_file_name):
        self.dose = group_dir
        self.readings = []
        self.integrals = []
        self.mean_integral = 0
        # os.walk -> dir: str com o diretório atual da iteração; subdirs: lista com os subdiretórios do diretório atual; files: lista com os arquivos do diretório atual
        for dir, subdirs, files in os.walk(group_dir):
            for file in files:
                if (file == osl_response_file_name):
                    _fp = os.path.abspath(os.path.join(dir, file))
                    _osl_reading = osl_response(file_path=_fp)
                    self.readings.append(_osl_reading)
                    self.integrals.append(_osl_reading.total_counts())
        self.mean_integral = np.mean(self.integrals)
        self.stdv = np.std(self.integrals)
    
    def plot(self, reading_number='all', filter=True, label=False):
        rd = self.readings
        fig = plt.figure(figsize=(20,10))
        axis = fig.add_subplot(111)
        axis.set_title('Resposta OSL para {}'.format(rd[0].dose))
        axis.set_xlabel('Tempo (s)')
        axis.set_ylabel('Luminescência (u.a.)')
        axis.legend(loc='best')
        if filter:
            if reading_number == 'all':
                for _ in rd:
                    lab = 'Dose = {} - Integral={}'.format(_.pastilha, _.total_counts()) if label else ''
                    axis.plot(_.savgol_filter(51,3)[0], _.savgol_filter(51,3)[1], label = lab)
                    axis.legend()
            elif len(rd) >= reading_number:
                lab = 'Dose = {} - Integral={}'.format(rd[reading_number].total_counts(), rd[reading_number].total_counts()) if label else ''
                axis.plot(rd[reading_number].savgol_filter(51,3)[0], rd[reading_number].savgol_filter(51,3)[1], label=lab)
                axis.legend()
        else:
            if reading_number == 'all':
                for _ in rd:
                    lab = 'Dose = {} - Integral={}'.format(_.pastilha, _.total_counts()) if label else ''
                    axis.plot(_.df.loc[:,'Tempo (s)'], _.df.loc[:,'Intensidade (u.a.)'], label = lab)
                    axis.legend()
            elif len(rd) >= reading_number:
                lab = 'Dose = {} - Integral={}'.format(rd[reading_number].pastiha, rd[reading_number].total_counts()) if label else ''
                axis.plot(rd[reading_number].df.loc[:,'Tempo (s)'], rd[reading_number].df.loc[:,'Intensidade (u.a.)'], label = lab)
                axis.legend()

In [38]:
# Equações
data = {}
for _ in OSL_RESPONSE_BASE_DIR: data[_] = osl_response_group(os.path.join(OSL_ROOT_DIR,_), osl_response_file_name=OSL_FILE_NAME)
slope, intercept, r, p, std_err = stats.linregress(IRRADIATED_DOSES, [data[_].mean_integral for _ in data])
def eq_cal(x):
  return slope * x + intercept
#plt.plot(IRRADIATED_DOSES, list(map(eq_cal, IRRADIATED_DOSES))
if LID:
  global lid
  lid = (np.mean(data['limpos'].mean_integral) + 3*np.std(data['limpos'].mean_integral))*(1/slope)
reproducibility = [(data[_].stdv/(math.sqrt(len(data[_].readings))*data[_].mean_integral))*100 for _ in data]

file_exsists = os.path.exists('resultados.txt')
if not os.path.exists('resultados.txt'):
  if interactive:
    file_name = str(input('Nome do arquivo com os resultados: '))
    file = open(file_name, 'a')
  else:
    file = open('resultados.txt', 'a')
  file.write('Equação de Calibração:\n\t\tContagem = Dose * {} + {}\n'.format(slope, intercept))
  file.write('Equação de Calibração:\n\t\tDose = Contagem*{} - {}\n'.format(1/slope, intercept*(1/slope)))
  if LID: file.write('LID = {}\n'.format(lid))
  file.write('r² = {}\n'.format(r**2))
  file.close()
else:
  print('O Arquivo já existe.')


print(data['0,1 Gy'].readings[0].dose)

O Arquivo já existe.


In [ ]:
# Gráficos
def set_fig():
    global fig
    global ax
    fig = plt.figure(num=1, figsize=(10,8), dpi=None, facecolor=None, edgecolor=None, frameon=True,clear=False)
    ax = fig.add_subplot(111)
    ax.set_xlabel('Tempo (s)', fontsize=15)
    ax.set_ylabel('Resposta LOE (u.a.)', fontsize=15)
    ax.grid()
    plt.xticks(fontsize=15)
    plt.yticks(fontsize=15)

def plot_dr(errorbars=False, save = ''):
    mean_integrals = [data[_].mean_integral for _ in OSL_RESPONSE_BASE_DIR]
    mean_integrals_std = [data[_].stdv for _ in OSL_RESPONSE_BASE_DIR]
    if not errorbars:
        ax.scatter(IRRADIATED_DOSES, mean_integrals, s=200, alpha=1)
        if save != '':
            plt.savefig(save, bbox_inches='tight', dpi=100)
    else:
        ax.errorbar(IRRADIATED_DOSES, mean_integrals, yerr=mean_integrals_std)

def plot_osl_response():
    for _ in data:
        data[_].plot(filter=True,reading_number=0, label=True)

data['0,1 Gy'].plot(filter=True,reading_number='all', label=True)
plt.savefig("teste.png", bbox_inches='tight', dpi=100)